# YOLOv8

In [7]:
import os
import argparse
import logging
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image

## Download Images from Bucket

In [ ]:
import sys
sys.path.append('../src')
from data_utils import download_images_and_labels


# Bucket and directory
bucket_name = 'tree-counting-project'
prefix = 'Dataset 348-17-15/' # using Roboflow dataset with 348 train, 17 val and 15 test images
local_dir = '../Data/Dataset 348-17-15'

# Download images
download_images_and_labels(bucket_name, prefix, local_dir)

## Installing YOLO

In [1]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 14918MiB)
Setup complete ✅ (1 CPUs, 3.6 GB RAM, 41.2/98.2 GB disk)


## Training

In [9]:
# Function for logging
def setup_logging(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    logging.basicConfig(filename=os.path.join(log_dir, 'log.txt'),
                        filemode='a',
                        format='%(message)s',
                        level=logging.DEBUG)
    logger = logging.getLogger()
    return logger


In [10]:
# function to plot training and val loss curves
def plot_metrics(log_dir, metrics):
    epochs = range(len(metrics['train_loss']))
    
    plt.figure()
    plt.plot(epochs, metrics['train_loss'], label='Train Loss')
    plt.plot(epochs, metrics['val_loss'], label='Val Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(log_dir, 'loss_curve.png'))
    plt.close()
    
    plt.figure()
    plt.plot(epochs, metrics['train_map50'], label='Train mAP50')
    plt.plot(epochs, metrics['val_map50'], label='Val mAP50')
    plt.title('mAP50 Curves')
    plt.xlabel('Epochs')
    plt.ylabel('mAP50')
    plt.legend()
    plt.savefig(os.path.join(log_dir, 'map50_curve.png'))
    plt.close()


In [11]:
def train_yolo(model_name, data_path, total_epochs, img_size, log_dir):
    # Load model
    model = YOLO(model_name)

    # Initialize metrics dictionary
    metrics = {'train_loss': [], 'val_loss': [], 'train_map50': [], 'val_map50': []}
    
    # Setup logging
    logger = setup_logging(log_dir)

    # Train model for one epoch
    results = model.train(
        data=data_path,
        epochs=total_epochs,
        imgsz=img_size,
        plots=False,
        save=True
    )
        
    # Extract metrics
    train_loss = results.metrics['train']['loss'][0]  # assuming loss is stored in the first index
    val_loss = results.metrics['val']['loss'][0]  # assuming loss is stored in the first index
    train_map50 = results.metrics['train']['map50'][0]  # mAP at IoU=0.50 for training
    val_map50 = results.metrics['val']['map50'][0]  # mAP at IoU=0.50 for validation

    # Store metrics
    metrics['train_loss'].append(train_loss)
    metrics['val_loss'].append(val_loss)
    metrics['train_map50'].append(train_map50)
    metrics['val_map50'].append(val_map50)

    # Log metrics
    log_entry = {
        'epoch': epoch,
        'train_loss': train_loss,
        'val_loss': val_loss,
        'train_map50': train_map50,
        'val_map50': val_map50,
        'n_parameters': model.model.n_params()
    }
    logger.info(json.dumps(log_entry))

    # Save checkpoint
    model.save(f'{log_dir}/checkpoint_epoch_{epoch}.pt')

    # Plot metrics
    plot_metrics(log_dir, metrics)

In [12]:
# argument parsing
def parse_args():
    parser = argparse.ArgumentParser(description="Train YOLOv8 model.")
    parser.add_argument('--model', type=str, default='yolov8s.pt', help='Pre-trained model path')
    parser.add_argument('--data', type=str, required=True, help='Path to dataset')
    parser.add_argument('--epochs', type=int, default=25, help='Number of epochs to train')
    parser.add_argument('--img_size', type=int, default=800, help='Image size for training')
    parser.add_argument('--log_dir', type=str, default='./logs', help='Directory to save logs and plots')
    return parser.parse_args()

def parse_args_notebook():
    args = argparse.Namespace(
        model='yolov8s.pt',
        data='/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/data.yaml',
        epochs=100,
        img_size=640,
        log_dir='./logs'
    )
    return args


In [ ]:
args = parse_args_notebook()
train_yolo(args.model, args.data, args.epochs, args.img_size, args.log_dir)

New https://pypi.org/project/ultralytics/8.2.49 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=Fals

train: Scanning /home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/train/labels.cache... 348 images, 0 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 368, len(boxes) = 29937. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning /home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/valid/labels.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to runs/detect/train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     80/100      6.21G      1.461     0.7048      1.132       1733        640: 100%|██████████| 22/22 [00:13<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.30s/it]

                   all         17       1130      0.858      0.804      0.844      0.386



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     81/100      4.43G      1.387     0.6735       1.11       1843        640:   5%|▍         | 1/22 [01:20<28:16, 80.77s/it]

In [17]:
args = parse_args_notebook()

# Put in arguments for training
data_yaml_path = '/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/data.yaml'
args.epochs = 100
args.img_size = 640
args.log_dir = '/home/jupyter/ee_tree_counting/Training/YOLOv8'


In [2]:
from ultralytics import YOLO

# Simplified YOLO training command
model = YOLO('yolov8s.pt')
results = model.train(
    data='/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/data.yaml',  # absolute path
    epochs=1,
    imgsz=640,
    plots=False,
    save=True,
    save_dir='ee_tree_counting/Training/YOLOv8'
)

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_f

train: Scanning /home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/train/labels.cache... 348 images, 0 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 368, len(boxes) = 29937. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning /home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/valid/labels.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to runs/detect/train7
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      6.32G      2.686      2.403      1.848       1707        640: 100%|██████████| 22/22 [00:14<00:00,  1.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.09s/it]

                   all         17       1130       0.42      0.768       0.68      0.253



1 epochs completed in 0.005 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 22.5MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.56it/s]

                   all         17       1130      0.419       0.77      0.682      0.253
Speed: 0.2ms preprocess, 6.1ms inference, 0.0ms loss, 2.6ms postprocess per image


In [10]:
import torch
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)


True
True


In [1]:
import torch

# Print PyTorch version
print("PyTorch version:", torch.__version__)

# Print CUDA version PyTorch was built with
print("CUDA version PyTorch was built with:", torch.version.cuda)

# Check if CUDA is available
print("Is CUDA available:", torch.cuda.is_available())

# Print the version of CUDA installed on the system
from subprocess import check_output
cuda_version = check_output("nvcc --version", shell=True).decode("utf-8")
print("CUDA version installed on system:\n", cuda_version)

# Print cuDNN version
print("cuDNN version:", torch.backends.cudnn.version())


PyTorch version: 1.13.1+cu117
CUDA version PyTorch was built with: 11.7
Is CUDA available: True
CUDA version installed on system:
 nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0

cuDNN version: 8900
